# Scrabble Project

Source: MITEdx Introduction to Computation/Python

The rules of the game are as follows:

## Dealing
A player is dealt a hand of n letters chosen at random (assume n=7 for now).

The player arranges the hand into as many words as they want out of the letters, using each letter at most once.

Some letters may remain unused (these won't be scored).

## Scoring
The score for the hand is the sum of the scores for each word formed.

The score for a word is the sum of the points for letters in the word, multiplied by the length of the word, plus 50 points if all n letters are used on the first word created.

Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is worth 3, D is worth 2, E is worth 1, and so on. We have defined the dictionary SCRABBLE_LETTER_VALUES that maps each lowercase letter to its Scrabble letter value.

For example, 'weed' would be worth 32 points ((4+1+1+2) for the four letters, then multiply by len('weed') to get (4+1+1+2)*4 = 32). Be sure to check that the hand actually has 1 'w', 2 'e's, and 1 'd' before scoring the word!

As another example, if n=7 and you make the word 'waybill' on the first try, it would be worth 155 points (the base score for 'waybill' is (4+1+4+3+1+1+1)*7=105, plus an additional 50 point bonus for using all n letters).

### Helper Code

In [1]:
import random
import string

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7
hand = {}
lastHand = {}
SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}


WORDLIST_FILENAME = "words.txt"

In [2]:
def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    inFile.close()
    return wordList

In [3]:
def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq
	

### Scoring a word

In [4]:
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    score = 0
    for letter in word:
        score += SCRABBLE_LETTER_VALUES[letter]
    score *= len(word)
    if len(word) == n:
        score += 50
    return score


### Displaying a Hand

In [5]:
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    outHand = ""
    for letter in hand.keys():
        for j in range(hand[letter]):
             outHand += letter + " "   
    app.hand_lbl.configure(text = "    Your Hand:  " + outHand)

### Deal hand

In [6]:
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    numVowels = n // 3
    
    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
        
    for i in range(numVowels, n):    
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
        
    return hand

### Update a hand by removing letters

In [7]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    tempHand = hand.copy()
    for letter in word:
        if tempHand[letter] >1:
            tempHand[letter] -= 1
        else:
            del tempHand[letter]
    return tempHand

### Test a word's validity

In [8]:
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    if word not in wordList:
        return False
    letterFreq = getFrequencyDict(word)
    for letter in letterFreq:
            if hand.get(letter,0) < letterFreq[letter]:
                return False
    return True
        

### Calculate hand len

In [9]:
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string-> int)
    returns: integer
    """
    count = 0 
    for letter in hand.keys():
        for j in range(hand[letter]):
            count += 1
    return count

### Play Hand

In [10]:
### Playing a game

In [11]:
def initialize():
    """ initialize Frame widgets"""
    app.score_lbl.configure(text = "    Hand Score:")            
    app.word.config(state = 'normal')
    app.word.delete(0, END)
    app.word.config(state = 'disable')
    global total
    total = 0
    

def startHand():
    """ Display a new hand and enable user entry"""
    displayHand(hand)
    #Allow user to enter a word
    app.word.config(state = 'normal')


def playHand():
    """When the user enters text and hit Enter, the Enter listener executes this function, 
       to process the word the user entered"""

    global hand
    global total
    word = app.word.get().strip().lower()
    app.word.delete(0, END)
    app.word.config(state = 'disable')
    if word != ".":
        if not isValidWord(word, hand, wordList):
            #allow user to enter another word
            app.word.config (state = 'normal')
            app.word.insert (0, "Incorrect word, try again")
            app.word.config (state = 'disable')
            app.after(750, wordDel)
        # Otherwise (the word is valid):
        else:
            # Tell the user the updated total score
            wordScore = getWordScore(word, calculateHandlen(hand))
            total += wordScore
            app.score_lbl.configure(text = "    Hand Score:  " + str(total))
            if len(word) < calculateHandlen(hand):   
                hand = updateHand(hand, word)
                displayHand(hand)
                app.word.config (state = 'normal')
            else:
                wordOver()
    else:
        wordOver()

def wordDel():
    app.word.config (state = 'normal')
    app.word.delete (0, END)    

            
def wordOver():
    """Word Game is over by user request of by using up all letters"""
    app.hand_lbl.configure(text = "    Draw a hand")
    app.create_buttons(False)
    app.submit_bttn.config(state = 'normal')
        
def playGame():
    """ When user hits the Submit button, the listener calls this function to 
        start the Game with playMode selection"""       

    global lastHand
    global hand
    choice = app.playMode.get()
    app.submit_bttn.config (state = 'disable')
    if choice == "Play New Hand":
        initialize()
        app.hand_lbl.configure(text = "    Draw a hand")
        hand = dealHand(HAND_SIZE)
        lastHand = hand.copy()
        startHand()
    elif choice == "Replay Last Hand":
        initialize()
        app.hand_lbl.configure(text = "    Draw a hand")
        hand = lastHand.copy()
        startHand()
    elif choice == "Exit Game":
        initialize()
        app.hand_lbl.configure(text = "         GAME OVER")
        app.word.config(state = 'disable')
        app.after(500, root.destroy)

In [12]:
#demonstrates scrabble game using a graphic interface with tkinter package

from tkinter import *

class Application(Frame):
    """Game of scrabble graphic application"""
    
    
    def __init__(self, master):
        """Initialize the frame."""
        super(Application, self).__init__(master)
        self.grid()
        self.create_widgets()
        self.create_buttons(True)
    
    def create_widgets(self):
        """Create buttons, text and entry widgets. """
        #create label for Game title
        self.inst_lbl = Label(self, text = "Game of Scrabble!", font='Helvetica 18 bold')
        self.inst_lbl.grid(row = 0, column = 0, columnspan =2, sticky = W)
        
        #create submit button
        self.submit_bttn = Button(self, text = "Submit", font= 'Helvetica 16 bold', command = self.playStart)
        self.submit_bttn.grid(row = 5, column = 0, sticky = W)       

        #create label for Hand
        self.hand_lbl = Label(self, text = "    Draw a hand" , font= 'Helvetica 16 bold')
        self.hand_lbl.grid(row = 8, column = 0, sticky = W)

        #create label for score
        self.score_lbl = Label(self, text = "    Hand Score:  ", font= 'Helvetica 16 bold')
        self.score_lbl.grid(row = 9, column = 0, sticky = W)       
        
        #create label for word
        self.word_lbl = Label(self, text = "    Enter word or . to Quit", font= 'Helvetica 16 bold')
        self.word_lbl.grid(row = 10, column = 0, sticky = W)
        #create entry to accept word on return
        self.word = Entry(self, font= 'Helvetica 16 bold')
        self.word.grid(row = 10, column = 1, sticky = W)
        self.word.bind("<Return>", self.processHand)
        self.word.config(state = 'disable')
    
        
    
    def create_buttons(self, firstHand):
        """ create two buttons if first Hand choices otherwise 3 buttons"""
        if firstHand:
            buttonChoice = ["Play New Hand", "Exit Game"]
        else:
            buttonChoice = ["Play New Hand", "Replay Last Hand", "Exit Game"]
        #create variable for mode buttons
        self.playMode = StringVar()
        self.playMode.set(None)
        row = 2
        column = 0
        #create button depending based choice
        for element in buttonChoice:
            Radiobutton(self,
                    text = element,
                    variable = self.playMode,
                    value = element,
                    ).grid(row = row, column = column, sticky = W)
            row += 1

    def processHand(self, event):
        """Listener to Enter in the text entry widget"""
        playHand()
   
    def playStart(self):
        """Listener to submit button"""
        playGame()

#Main program: Load disctionary and start main loop for graphic interface
wordList = loadWords()
root = Tk()
root.title("Game of Scrabble")
app = Application(root)
root.mainloop()
    



Loading word list from file...
   83667 words loaded.
